<a href="https://colab.research.google.com/github/sergeymasl/my_lifehacks/blob/main/Airflow_base_of_knowledge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Connections

Connections представляют собой подключения к разным источникам данных, задаются они в веб интерфейсе, с указанием `conn_id`. Это `conn_id` в последствии можно вызвывать в тасках

При необходимости к Connections можно получить к данному объекту мы можем обращаться как к обычному словарю.

[ссылка](https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/hooks/base/index.html)

```python
# Пример доступа к переменной connection
from airflow.hooks.base_hook import BaseHook
host = BaseHook.get_connection("postgres_default").host
pass = BaseHook.get_connection("postgres_default").password
```

#### Variables

Глобальные переменные которые также могут задаваться в веб интерфейсе.

Доступ к переменным осуществляется через `key`.
Глобальные переменные можно:
- получать - `get`
- записывать - `set`
- обновлять - `update`
- удалять - `delete`

[ссылка](https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/models/variable/index.html)

```python
# Пример доступа к глобальной переменной
from airflow.models import Variable
foo = Variable.get(key = "key")
# присер записи глобальной переменной
a_dict = {'login' : con}
Variable.set(key = 'test_key', value = a_dict, serialize_json = True)
```